In [ ]:
import torch
from torchvision import transforms
import timm
from PIL import Image
from tqdm import tqdm

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.5 MB/s eta 0:00:00


In [ ]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)

    return input_batch


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
def predict(model, input_tensor):
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)

    return output

image_path = '/content/drive/MyDrive/ondc/Catalog Digitization/ONDC Test Data _ Images/Product Images/7up_Cool_Drink_Pet.jpg'
input_batch = preprocess_image(image_path)
output = predict(model, input_batch)


In [ ]:
output.shape

torch.Size([1, 1000])

In [ ]:
import os
folderpath = "/content/drive/MyDrive/ondc/Catalog Digitization/ONDC Test Data _ Images/Product Images"
feat = {}
for i in tqdm(range(len(os.listdir(folderpath)))):
  imagepath = os.path.join(folderpath,os.listdir(folderpath)[i])
  inputbatch = preprocess_image(imagepath)
  output = predict(model,inputbatch)
  feat[os.listdir(folderpath)[i]]=output

100%|██████████| 1060/1060 [13:20<00:00,  1.32it/s]


In [ ]:
feat

In [ ]:
import pickle

my_dict = feat
file_path = '/content/featureswithnames.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(my_dict, file)

print(f'Dictionary saved to {file_path}')


Dictionary saved to /content/featureswithnames.pkl
